# Chain-of-Thought - Forcing the LLM to "Show Its Work"

Hello everyone. Let's dive into another impactful prompting technique: **Chain-of-Thought (CoT) Prompting**.

The core idea is to counteract the model's natural tendency to "rush" to an answer. For any problem that requires logic, arithmetic, or multi-step reasoning, forcing the model to slow down and "show its work" dramatically improves its accuracy. This is particularly effective when working with lower-end LLMs (less reasoning capabilities).

This is done by explicitly instructing the model to think step-by-step. This simple instruction creates a "chain" of reasoning where each thought builds on the last, leading to a more reliable conclusion.

## The Task and Helper Functions

First, let's set up our helper functions and context data.

In [ ]:
import inspect
import litellm
from IPython.display import display, Markdown
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

# --- Configuration & Context ---
MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 500 # More tokens for reasoning + code

# --- The "messy" code to be refactored ---
def get_user_initials(user_data_list):
    initials_list = []
    for user in user_data_list:
        if 'name' in user and len(user['name']) > 0:
            parts = user['name'].split(' ')
            initials = ''
            for part in parts:
                initials += part[0].upper()
            initials_list.append(initials)
    return initials_list

CODE_TO_REFACTOR = inspect.getsource(get_user_initials)

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content


print("Setup complete. Helper functions and code context are ready.")

## Attempt 1: No Chain-of-Thought

Let's first ask the model for a direct answer without guiding its thought process. This demonstrates the model's default "fast thinking" mode, which often produces only the final result and does not provide much information into how the model tackled the problem.

In [ ]:
prompt_no_cot = [
    {
        "role": "user",
        "content": dedent(f"""
        Please refactor the following Python code:

        ```python
        {CODE_TO_REFACTOR}
        ```
        """)
    }
]

response_no_cot = get_completion(prompt_no_cot)

In [ ]:
display(Markdown(response_no_cot))

## Attempt 2: Introducing Chain-of-Thought

For maximum reliability on complex, repeatable tasks, we can use **Chain-of-Thought**, which can become even more powerful when combined with a couple few-shot examples.

In [ ]:
prompt_with_cot = [
    {
        "role": "user",
        "content": dedent(f"""
        Please refactor the following Python code.
        Let's think step by step and please provide your reasoning.
        
        While solving the problem, consider, among others, documentation, typing, readibility, etc.

        ```python
        {CODE_TO_REFACTOR}
        ```
        """)
    }
]

response_with_cot = get_completion(prompt_with_cot)

In [ ]:
display(Markdown(response_with_cot))